# Overview

This is meant as a supporting document for Sherry and Emmit's group meeting about basic git control, package management and testing. 

## First steps - building a new repo

In general there are two ways to set up a new git repo. If you have some code that you would like to turn into a fresh repo then you can start things from the command line. If you have a plan for a new repo but haven't done anything yet, it's likely easier to start from the github website and clone into an empty repo that you set up there. We'll start with the command line repo from an existing repository. In this case you can begin by initializing git and adding everything locally. 


In [ ]:
!git init
!git add .
!git commit -m "First commit"
!git branch -M main

This will initialize local git management and make the first commit for the new project. After this we can create a new github repository and push to that. 

In [ ]:
#!git remote add origin $YOUR_GIT_REPO_URL$ #for example: https://github.com/sherryli59/Ising_demo.git
!git fetch origin
!git remote add origin https://github.com/sherryli59/Ising_demo.git

!git branch --set-upsRream-to=origin/main main
!git push -u origin main

These commands will create the new remote (changing to your preferred location) and push existing work to that file. 

### Starting from GitHub

If you have to start from github, then you will initialize your directory and locally run 


In [ ]:
!git clone https://github.com/sherryli59/Ising.git

Since this is simpler, this is generally preferred for new projects. In either method, at this point we can begin using git to manage our changes as desired. 

## Building a package

One useful way to manage our code once we have it built is to create a package that we can install locally. This can be done by writing a setup.py file (an example can be found in the ising projects root directory). Once this is done we can build the package for use locally. This will also reflect any changes in the underlying code in python so there's no need to reinstall the package if we change the code. To install the package we should create a new conda directory as

In [ ]:
!conda create -n ising -y
!conda init
!conda activate ising

It's helpful to build a package out of our project from the very beginning so we don't have to deal with relative import drama. One way to do this is through a setup.py file. We can install the package using pip and dependencies will automatically be added to our conda environment. Note that the -e flag is to make sure the package is in an editable state so that so you don't have to re-install the package every time you edit it locally.

In [ ]:
!pip install -e .
# alternatively, you can run `!sudo python setup.py develop clean` to install the package in development mode

# Other useful git features

One good option to make git usage more consistent is to use pre-commit hooks. These are local instructions that will be executed whenever a commit is made. They can be stored in .git/hooks/pre-commit (there is an existing file called pre-commit.sample that provides basic funcitonality). One of the better usages of pre-commit hooks is to automatically use a code formatting software such as black or autopep8. An example of a commit to automatically apply black to python files is below. 

In [ ]:
!/bin/bash

# Find all Python files in the repository
python_files=$(git ls-files | grep '\.py$')

# Check if there are any Python files
if [ -n "$python_files" ]; then
    # Run black on each Python file
    for file in $python_files; do
        black --check "$file"
        # Check if black modified the file
        if [[ $? -ne 0 ]]; then
            echo "Error: black failed to run on $file"
            exit 1
        fi
    done
fi

# Continue with the commit
exit 0

Having multiple branches in a Git repository enables isolation of Features: Each branch can represent a separate feature, bug fix, or experiment. We can work on these features independently without affecting the main codebase. In the next exercise, we want to duplicate the main branch into another branch that implements lattice gas instead. To do this, we can run

In [ ]:
!git checkout -b lattice_gas main